In [31]:
import numpy as np
import os

In [32]:
eval_folder = "/checkpoints/sticker/DPO/eval-test"
res_folder = []
for subf in os.listdir(eval_folder):
    if subf.startswith("checkpoint"):
        res_folder.append(subf)

In [33]:
score_name = "select_scores_dm_scale7.0.npy"

In [34]:
all_select_scores = {}
for folder in res_folder:
    path = os.path.join(eval_folder, folder, score_name)
    select_scores = np.load(path, allow_pickle=True).item()
    all_select_scores[folder] = select_scores

In [35]:
example_key = list(sorted(all_select_scores.keys(), key=lambda x:float(x.split("-")[-1][4:])))[0]

In [37]:
example_key

'checkpoint-375-scale1.0-mask0.0'

In [38]:
select_ratio = 0.5

In [39]:
re_select_scores = {}
for uid in example_scores:
    if uid not in re_select_scores:
        re_select_scores[uid] = {}

    for iid in example_scores[uid]:
        if iid not in re_select_scores[uid]:
            re_select_scores[uid][iid] = {}

        for gid in example_scores[uid][iid]:
            if gid not in re_select_scores[uid][iid]:
                re_select_scores[uid][iid][gid] = {}
                re_select_scores[uid][iid][gid]["folder"] = []
                re_select_scores[uid][iid][gid]["seq_id"] = []
                re_select_scores[uid][iid][gid]["img_id"] = []
                re_select_scores[uid][iid][gid]["score"] = []
            
            for folder in all_select_scores:
                cur_scores = all_select_scores[folder][uid][iid][gid]
                all_seq_id = []
                all_img_id = []
                all_seq_score = []
                for seq_id in cur_scores:
                    all_seq_id.append(seq_id)
                    tgt_scores = cur_scores[seq_id]["tgt_score"]
                    hist_scores = cur_scores[seq_id]["hist_score"]
                    seq_scores = [select_ratio * tgt_score + (1 - select_ratio) * hist_score for tgt_score,hist_score in zip(tgt_scores, hist_scores)]
                    max_score = max(seq_scores)
                    max_img_idx = seq_scores.index(max_score)
                    all_img_id.append(max_img_idx)
                    all_seq_score.append(max_score)
                
                max_score = max(all_seq_score)
                max_idx = all_seq_score.index(max_score)
                max_seq_id = all_seq_id[max_idx]
                max_img_id = all_img_id[max_idx]

                re_select_scores[uid][iid][gid]["folder"].append(folder)
                re_select_scores[uid][iid][gid]["seq_id"].append(max_seq_id)
                re_select_scores[uid][iid][gid]["img_id"].append(max_img_id)
                re_select_scores[uid][iid][gid]["score"].append(max_score)

In [40]:
from copy import deepcopy

In [41]:
final_select_scores = deepcopy(re_select_scores)

for uid in re_select_scores:
    for iid in re_select_scores[uid]:
        for gid in re_select_scores[uid][iid]:
            scores = re_select_scores[uid][iid][gid]["score"]
            best_score = max(scores)
            best_idx = scores.index(best_score)
            best_folder = re_select_scores[uid][iid][gid]["folder"][best_idx]
            best_seq_id = re_select_scores[uid][iid][gid]["seq_id"][best_idx]
            best_img_id = re_select_scores[uid][iid][gid]["img_id"][best_idx]

            final_select_scores[uid][iid][gid]["folder"] = best_folder
            final_select_scores[uid][iid][gid]["seq_id"] = best_seq_id
            final_select_scores[uid][iid][gid]["img_id"] = best_img_id
            final_select_scores[uid][iid][gid]["score"] = best_score

In [42]:
np.save(os.path.join(eval_folder, "final_select_scores.npy"), np.array(final_select_scores))

In [43]:
all_res = {}
res_name = "all_res_dm_scale7.0.npy"
for folder in res_folder:
    path = os.path.join(eval_folder, folder, res_name)
    res = np.load(path, allow_pickle=True).item()
    all_res[folder] = res

In [44]:
import shutil
final_folder = example_key.split("-")
final_folder[-1] = "mask2.0"
final_folder = "-".join(final_folder)

final_res = deepcopy(all_res[example_key])
for uid in final_select_scores:
    for iid in final_select_scores[uid]:
        for gid in final_select_scores[uid][iid]:
            folder = final_select_scores[uid][iid][gid]["folder"]
            seq_id = final_select_scores[uid][iid][gid]["seq_id"]
            img_id = final_select_scores[uid][iid][gid]["img_id"]
            score = final_select_scores[uid][iid][gid]["score"]

            seqs_info = {}
            seqs_info[0] = deepcopy(all_res[folder][uid][iid][gid]["seqs"][seq_id])
            seqs_info[0]["src_images"] = seqs_info[0]["images"][img_id:img_id+1]
            seqs_info[0]["images"] = []

            for src_img_path in seqs_info[0]["src_images"]:
                src_img_folder = "/".join(src_img_path.split("/")[:-2])
                img_path_list = [final_folder] + src_img_path.split("/")[1:]
                tgt_img_folder = "/".join(img_path_list[:-2])
                os.makedirs(os.path.join(eval_folder, tgt_img_folder), exist_ok=True)

                for item in os.listdir(os.path.join(eval_folder, src_img_folder)):
                    src_path = os.path.join(eval_folder, src_img_folder, item)
                    if os.path.isfile(src_path):
                        shutil.copy(src_path, os.path.join(eval_folder, tgt_img_folder))

                tgt_img_folder = "/".join(img_path_list[:-2] + ["0"])
                os.makedirs(os.path.join(eval_folder, tgt_img_folder), exist_ok=True)
                tgt_img_path = "/".join(img_path_list[:-2] + ["0", "0.jpg"])
                shutil.copy(os.path.join(eval_folder, src_img_path), os.path.join(eval_folder, tgt_img_path))  

                seqs_info[0]["images"].append(tgt_img_path)

            final_res[uid][iid][gid]["seqs"] = seqs_info

In [45]:
np.save(os.path.join(eval_folder, final_folder, res_name), np.array(final_res))